In [ ]:
import oracledb
import pandas as pd
from google.cloud.bigquery import Client
import os, json
from google.cloud import secretmanager
import pandasql as ps

In [ ]:
def set_secrets_as_envs():
  secrets = secretmanager.SecretManagerServiceClient()
  resource_name = f"{os.environ['KNADA_TEAM_SECRET']}/versions/latest"
  secret = secrets.access_secret_version(name=resource_name)
  secret_str = secret.payload.data.decode('UTF-8')
  secrets = json.loads(secret_str)
  os.environ.update(secrets)

In [ ]:
def oracle_secrets():
  set_secrets_as_envs()
  return dict(
    user=os.getenv('DB_USER'),
    password=os.getenv('DB_PASSWORD'),
    host = os.getenv('DBT_ORCL_HOST'),
    service = os.getenv('DBT_ORCL_SERVICE'),
    project_id = os.getenv('BS_PROJECT_ID'),
    table_uri = os.getenv('BS_TABLE_URI'),  
    encoding="UTF-8",
    nencoding="UTF-8"
  )

oracle_secrets = oracle_secrets()

In [ ]:
def get_data_from_table(max_id):
    PROJECT_ID = oracle_secrets['project_id'] 
    TABLE_URI = oracle_secrets['table_uri'] 

    client = Client(project=PROJECT_ID)
    job = client.query(f"SELECT * FROM `{TABLE_URI}` WHERE id > {max_id}")
    df = job.to_dataframe()
    
    # some data cleaning
    df['belop'] = pd.to_numeric(df['belop'])
    
    char_to_replace = {'≥' : '>=', '≤' : '<='}
    for old, new in char_to_replace.items():
        df['sats_beskrivelse'] = df['sats_beskrivelse'].str.replace(old, new)
    
    return df

In [ ]:
def get_fk_person1_data(min_date, conn):
    sql = (f""" select * from dt_person.ident_off_id_til_fk_person1 
            where GYLDIG_FRA_DATO >= to_date('{min_date}', 'YYYY-MM-DD HH24:MI:SS')""")
    
    df = pd.read_sql(sql = sql, con = conn)
    return df

In [ ]:
def joined_df(df,df_):
    df = df
    df_ = df_
    
    query = ("""select id, fnr_barn, fnr_innsender, orgnr, bestillingsdato,
                        brillepris, bestillingsreferanse, behandlingsresultat, sats,
                        sats_belop, sats_beskrivelse, belop, b.fk_person1 fk_person1_barn, i.fk_person1 fk_person1_innsender,opprettet 
                        from df
                        left outer join df_ b on
                        df.fnr_barn=b.off_id
                        and b.gyldig_fra_dato<=df.bestillingsdato
                        and b.gyldig_fra_dato>=df.bestillingsdato
                        and b.skjermet_kode=0
                        left outer join df_ i on
                        df.fnr_barn=i.off_id
                        and i.gyldig_fra_dato<=df.bestillingsdato
                        and i.gyldig_fra_dato>=df.bestillingsdato""")

    fk_person1_df = ps.sqldf(query, locals())
    
    fk_person1_df = fk_person1_df[['id', 'fk_person1_barn','fk_person1_innsender', 'orgnr','bestillingsdato','brillepris', 
                'bestillingsreferanse', 'behandlingsresultat', 'sats','sats_belop', 'sats_beskrivelse', 'belop','opprettet']] 
    fk_person1_df['opprettet'] = fk_person1_df['opprettet'].astype('datetime64[ns]')
    fk_person1_df['bestillingsdato'] = pd.to_datetime(fk_person1_df['bestillingsdato'], format='%Y-%m-%d')
    
    return fk_person1_df

In [ ]:
def send_data():
    user = oracle_secrets['user'] + '[DVH_FAM_HM]'
    dsn_tns = oracledb.makedsn(oracle_secrets['host'], 1521, service_name = oracle_secrets['service'])
    
    with oracledb.connect(user=user, password = oracle_secrets['password'], dsn=dsn_tns) as conn:
        with conn.cursor() as cursor:
            sql = ("""select max(id) as maximun from brillestonad_test""")
            cursor.execute(sql)
            result = cursor.fetchall()
            max_id = result[0][0]
            
            if max_id is None:
                df = get_data_from_table(0)
            else:
                df = get_data_from_table(max_id)
                                
            if len(df) > 0:
                
                min_date = df['bestillingsdato'].min()
                df_ = get_fk_person1_data(min_date, conn)
                fk_person1_df = joined_df(df,df_)
                                
                rows = [tuple(x) for x in fk_person1_df.values]
                cursor.executemany('''INSERT INTO brillestonad_test(ID,FK_PERSON1_BARN,FK_PERSON1_INNSENDER,ORGNR,BESTILLINGSDATO,BRILLEPRIS,BESTILLINGSREFERANSE,
                                        BEHANDLINGSRESULTAT,SATS,SATS_BELOP,SATS_BESKRIVELSE,BELOP,OPPRETTET) 
                                        VALUES (:1,nvl(:2,-1),nvl(:3,-1),:4,:5,:6,:7,:8,:9,:10,:11,:12,:13)''',rows)
                conn.commit()

In [ ]:
send_data()